# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather = "weather_output.csv"
weather_df = pd.read_csv(weather)
weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,evensk,12,RU,1577487799,60,61.92,159.24,0.81,12.91
1,1,rikitea,90,PF,1577487809,86,-23.12,-134.97,77.47,13.69
2,2,posse,100,BR,1577487819,77,-14.08,-46.36,73.49,2.91
3,3,ushuaia,75,AR,1577487684,44,-54.81,-68.31,59.00,14.99
4,4,tuatapere,49,NZ,1577487839,74,-46.13,167.69,53.01,18.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [10]:
locations = weather_df[["Latitude", "Longitude"]]

humidity = weather_df["Humidity"].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
idealweather_df = weather_df.dropna()
idealweather_df = idealweather_df.loc[(weather_df['Max Temp']>70)&(weather_df['Max Temp']<80)]
idealweather_df = idealweather_df.loc[(weather_df['Wind Speed']<10)]
idealweather_df = idealweather_df.loc[(weather_df['Cloudiness']==0)]

idealweather_df.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
74,74,busselton,0,AU,1577488547,45,-33.64,115.35,72.00,2.39
146,146,youkounkoun,0,GN,1577489280,27,12.53,-13.12,73.13,0.58
180,180,tabou,0,CI,1577489631,85,4.42,-7.36,77.81,3.65
182,182,mahina,0,ML,1577489652,9,13.76,-10.85,73.56,5.70
273,273,vung tau,0,VN,1577490593,82,10.35,107.08,71.35,3.29


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = idealweather_df
hotel_df["Hotel Name"] = ""

params = {"radius": 5000, "types": "hotels", "keyword": "hotels", "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
74,74,busselton,0,AU,1577488547,45,-33.64,115.35,72.00,2.39,Busselton Ithaca Motel
146,146,youkounkoun,0,GN,1577489280,27,12.53,-13.12,73.13,0.58,
180,180,tabou,0,CI,1577489631,85,4.42,-7.36,77.81,3.65,Hotel Les Rochers
182,182,mahina,0,ML,1577489652,9,13.76,-10.85,73.56,5.70,Madi Sissoko
273,273,vung tau,0,VN,1577490593,82,10.35,107.08,71.35,3.29,The IMPERIAL Hotel
348,348,siocon,0,PH,1577491369,83,11.03,124.04,79.74,6.87,Hisoler's Beach Resort
411,411,lagos,0,NG,1577492021,94,6.46,3.39,78.80,2.24,Eko Hotels & Suites
437,437,beloha,0,MG,1577492289,71,-25.17,45.06,77.02,7.54,
439,439,jiwani,0,PK,1577492310,61,25.05,61.74,70.09,9.51,Victoria Hut
443,443,gwadar,0,PK,1577492352,48,25.12,62.33,71.64,6.13,Zaver Pearl Continental Hotel


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))